In [2]:
import tensorflow as tf
import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from keras.layers import Flatten,Dropout,Dense,BatchNormalization,Conv2D,MaxPooling2D,Conv2DTranspose,concatenate,Input
from keras.models import Model
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [5]:
def horizontalFlip(arr):
    arr1 = cv2.flip(arr,0)
    return arr1
def verticalFlip(arr):
    arr1 = cv2.flip(arr,1)
    return arr1
def rotate90(arr):
    arr1 = cv2.rotate(arr,cv2.ROTATE_90_CLOCKWISE)
    return arr1
def rotate180(arr):
    arr1 = cv2.rotate(arr,cv2.ROTATE_180)
    return arr1
def rotate270(arr):
    arr1 = cv2.rotate(arr,cv2.ROTATE_90_COUNTERCLOCKWISE)
    return arr1



In [6]:
inputs= list()
outputs = list()
initial_path = 'data/input/'
output_path = 'data/cups/'
max_images = 100+1
N=128
for i in range(1,max_images):
    

    curr_path = output_path + str(i) + '.tif'
    curr_image = cv2.imread(curr_path,cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(curr_image,(N,N))
    outputs.append(tf.convert_to_tensor(resized_image))
    curr1 = horizontalFlip(resized_image)
    curr2 = verticalFlip(resized_image)
    curr3 = rotate90(resized_image)
    curr4 = rotate180(resized_image)
    curr5 = rotate270(resized_image)
    outputs.append(tf.convert_to_tensor(curr1))
    outputs.append(tf.convert_to_tensor(curr2))
    outputs.append(tf.convert_to_tensor(curr3))
    outputs.append(tf.convert_to_tensor(curr4))
    outputs.append(tf.convert_to_tensor(curr5))
    
    
    curr_path = initial_path + str(i) + '.png'

    curr_image = cv2.imread(curr_path)
    resized_image = cv2.resize(curr_image,(N,N))
    inputs.append(tf.convert_to_tensor(resized_image))# numpy arrays convert to tensor
    
    curr1 = horizontalFlip(resized_image)
    curr2 = verticalFlip(resized_image)
    curr3 = rotate90(resized_image)
    curr4 = rotate180(resized_image)
    curr5 = rotate270(resized_image)
    inputs.append(tf.convert_to_tensor(curr1))
    inputs.append(tf.convert_to_tensor(curr2))
    inputs.append(tf.convert_to_tensor(curr3))
    inputs.append(tf.convert_to_tensor(curr4))
    inputs.append(tf.convert_to_tensor(curr5))
    


In [7]:
max_images = len(inputs)
print(max_images)
for index in range(1, max_images+1):
    inputs[index-1] = tf.divide(tf.cast(inputs[index-1], tf.float32), 255.0)
    outputs[index-1] = tf.divide(tf.cast(outputs[index-1], tf.float32), 255.0)

600


In [12]:
def createModel(input_size=(N, N, 3)):
    ins = tf.keras.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(4, 3, activation='relu', padding='same')(ins)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(8, 3, activation='relu', padding='same')(pool1)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = layers.Conv2D(16, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(16, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    concat1 = layers.Concatenate(axis=3)([conv2, up1])
    conv4 = layers.Conv2DTranspose(8, 2, activation='relu', padding='same')(concat1)

    conv4 = layers.Dropout(0.1)(conv4)

    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    concat2 = layers.Concatenate(axis=3)([conv1, up2])
    conv5 = layers.Conv2DTranspose(4, 2, activation='relu', padding='same')(concat2)
    
    drop = layers.Dropout(0.1)(conv5)

    # Output layer
    outs = layers.Conv2D(1, 1, activation='sigmoid')(drop)

    model = models.Model(inputs=ins, outputs=outs)
    return model
model = createModel()
model.compile(optimizer='adam', loss='binary_crossentropy')


model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)    │ (None, 128, 128, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_17 (Conv2D)            │ (None, 128, 128, 4)       │             112 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_8               │ (None, 64, 64, 4)         │               0 │ conv2d_17[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_18 (Conv2D)            │ (None, 64, 64, 8)         │             296 │ max_pooling2d_8[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_9               │ (None, 32, 32, 8)         │               0 │ conv2d_18[0][0]            │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_19 (Conv2D)            │ (None, 32, 32, 16)        │           1,168 │ max_pooling2d_9[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_20 (Conv2D)            │ (None, 32, 32, 16)        │           2,320 │ conv2d_19[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d_6               │ (None, 64, 64, 16)        │               0 │ conv2d_20[0][0]            │
│ (UpSampling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_6 (Concatenate)   │ (None, 64, 64, 24)        │               0 │ conv2d_18[0][0],           │
│                               │                           │                 │ up_sampling2d_6[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_transpose_4            │ (None, 64, 64, 8)         │             776 │ concatenate_6[0][0]        │
│ (Conv2DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_4 (Dropout)           │ (None, 64, 64, 8)         │               0 │ conv2d_transpose_4[0][0]   │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d_7               │ (None, 128, 128, 8)       │               0 │ dropout_4[0][0]            │
│ (UpSampling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_7 (Concatenate)   │ (None, 128, 128, 12)      │               0 │ conv2d_17[0][0],           │
│                               │                           │                 │ up_sampling2d_7[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_transpose_5            │ (None, 128, 128, 4)       │             19

 Total params: 4,873 (19.04 KB)

 Trainable params: 4,873 (19.04 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(inputs, outputs, test_size=0.1, random_state=41)
train_inputs = tf.stack(train_inputs, axis=0)
test_inputs = tf.stack(test_inputs, axis=0)
train_outputs = tf.stack(train_outputs, axis=0)
test_outputs = tf.stack(test_outputs, axis=0)

In [19]:
model.fit(train_inputs, train_outputs, epochs=10, validation_data=(test_inputs, test_outputs))

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.0025 - val_loss: 0.0027
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0025 - val_loss: 0.0028
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0025 - val_loss: 0.0028
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0025 - val_loss: 0.0028
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - loss: 0.0025 - val_loss: 0.0028
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - loss: 0.0025 - val_loss: 0.0027
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - loss: 0.0025 - val_loss: 0.0027
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - loss: 0.0026 - val_loss: 0.0027
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - loss: 0.0025 - val_loss: 0.0026
Epoch 10/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - loss: 0.0025 - val_loss: 0.0026


In [20]:
precision=0
recall=0
accuracy=0
F1=0
specificity=0
t_count=600
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
arr=np.zeros((N,N),dtype=np.uint8)

for i1 in range(t_count):
    print(i1)
    test_preds = model.predict(tf.expand_dims(inputs[i1], 0))
    test_preds[0]*=255
    for i in range(N):
        for j in range(N):
            arr[i][j]=test_preds[0][i][j][0]
    check_against=np.array(outputs[i1])
    arr = cv2.GaussianBlur(arr,(5,5),0)
    __,th = cv2.threshold(arr,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    TP = np.sum((th == check_against) & (th == 1))
    TN = np.sum((th == check_against) & (th == 0))
    FP = np.sum((th != check_against) & (th == 1))
    FN = np.sum((th != check_against) & (th == 0))
    
    precision += (TP) / (TP + FP)
    recall += (TP) / (TP + FN) 
    accuracy += (TP + TN) / (N * N)
    specificity += TN / (TN + FP)
precision/=t_count
recall/=t_count
accuracy/=t_count
F1 = (2 * precision * recall) / (precision + recall)
specificity/=t_count
print("Accuracy = ",accuracy)
print("recall = ",recall)
print("precision = ",precision)
print("specificity = ",specificity)
print("F1 = ",F1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
588
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
589
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
590
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
591
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
592
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
593
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
594
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
595
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
596
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
597
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
598
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
599
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Accuracy =  0.99907470703125
recall =  0.9511557559572335
precision =  0.8757868898651489
specificity =  0.9993408055761824
F1 =  0.9119166857174632


In [1]:
check_num=415
model=keras.saving.load_model('cups.keras')
test_preds = model.predict(tf.expand_dims(inputs[check_num], 0))
test_preds[0]*=255
arr=np.zeros((N,N),dtype=np.uint8)
for i in range(N):
    for j in range(N):
        arr[i][j]=test_preds[0][i][j][0]
arr = cv2.GaussianBlur(arr,(5,5),0)
__,th = cv2.threshold(arr,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
#th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=1)


#print(test_preds[0])
plt.imshow(inputs[check_num],'gray')
plt.show()
plt.imshow(th, cmap='gray')
plt.show()
plt.imshow(outputs[check_num],'gray')
plt.show()
#print(model_stats(model))

NameError: name 'keras' is not defined

In [21]:
model.save('new_cups.keras')